In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import cv2
from scipy import interpolate

def showImg(argc):
    f, axs = plt.subplots(1, 1, )
    axs.imshow(argc)
    axs.axis('off')

def getRaw(filename):
    img = matplotlib.image.imread(filename)
    return img

In [ ]:
rawImg = getRaw('banana_slug.tiff')
showImg(rawImg)

In [ ]:
#print('shape:', rawImg.shape)
#print('dtype: ', rawImg.dtype)
#print('size:', rawImg.size)
(pic_h, pic_w) = rawImg.shape
print('w: %d, h: %d\n' % (pic_w, pic_h))
# change raw format to float64

print('raw image pixel max value: %d, min value: %d\n' % (rawImg.max(), rawImg.min()))
rawImg[rawImg<2047]=2047
rawImg[rawImg>15000]=15000

rawImgDouble = rawImg.astype(np.float64)
#print('dtype: ', rawImgDouble.dtype)


In [ ]:
showImg(rawImgDouble)

In [ ]:
# map pixel value to [0.0...1.0]
def linearizationImg(img):
    imgArray = img.flatten()
    #print(imgArray)
    imgArray = (imgArray - 2047) / (15000 - 2047)
    #print(imgArray)
    #showImg(imgArray.reshape(pic_h, pic_w))
    #print(imgArray.reshape(pic_h, pic_w))
    return imgArray.reshape(pic_h, pic_w)
    

In [ ]:
linearImg = linearizationImg(rawImgDouble)
showImg(linearImg)

In [ ]:
# raw image rggb
def doGrayAWB(img):
    raw_r = img[0::2, 0::2]
    raw_gr = img[1::2, 0::2]
    raw_gb = img[0::2, 1::2]
    raw_b = img[1::2, 1::2]
    avg_r = np.mean(raw_r)
    avg_g = np.mean([raw_gr, raw_gb])
    avg_b = np.mean(raw_b)
    
    img[0::2, 0::2] = img[0::2, 0::2] * avg_g / avg_r
    img[1::2, 1::2] = img[1::2, 1::2] * avg_g / avg_b
    
    img[img>1.0] = 1.0
    
    #showImg(cv2.merge([img[1::2, 1::2], img[0::2, 1::2], img[0::2, 0::2]]))
    #img_gray = cv2.merge([img[0::2, 0::2], img[0::2, 1::2], img[1::2, 1::2]])
    #cv2.imwrite("./gray.jpg", img_gray)
    showImg(cv2.merge([img[0::2, 0::2], img[0::2, 1::2], img[1::2, 1::2]]))
    
    return img

In [ ]:
awbImg = doGrayAWB(linearImg)

In [ ]:
def demosaicAlgo(img):
    x = np.arange(0, pic_w)
    y = np.arange(0, pic_h)
    x1 = np.arange(0, pic_w, 2)
    y1 = np.arange(0, pic_h, 2)
    interpolate_func_r = interpolate.interp2d(x1, y1, img[0::2, 0::2])
    interpolate_func_g = interpolate.interp2d(x1, y1, (img[1::2, 0::2] + img[0::2, 1::2]) / 2)
    interpolate_func_b = interpolate.interp2d(x1, y1, img[1::2, 1::2])
    
    #showImg(cv2.merge([interpolate_func_r(x, y), interpolate_func_g(x, y), interpolate_func_b(x, y)]))
    
    return cv2.merge([interpolate_func_b(x, y), interpolate_func_g(x, y), interpolate_func_r(x, y)])

In [ ]:
rgbImg = demosaicAlgo(awbImg)

rgbImg[rgbImg > 1.0] = 1.0
K = (15000-2047) / 255
#print(rgbImg*(15000-2047)/K)
rgbImgInt = (rgbImg*((15000-2047) / K)*3.5).astype(np.int)
cv2.imwrite("./result_demosaic.jpg", rgbImgInt)

In [ ]:
def gammaCorrection(imgIn):
    gamma_ratio = 2.2
    
    img = imgIn.copy()
    isDarkPixel=imgIn<=0.0031308
    isBrightPixel=np.logical_not(isDarkPixel)
    img[isDarkPixel] = 12.92 * img[isDarkPixel]
    img[isBrightPixel] = (1 + 0.055) * (img[isBrightPixel]) ** (1 / gamma_ratio) - 0.055
    img = img * 255
    img = np.minimum(img, 255)
    return img


In [ ]:
gammaImg = gammaCorrection(rgbImg*2.0)
gammaImg = np.minimum(gammaImg, 255)
#showImg(gammaImg)
gammaImg = np.uint8(gammaImg)
cv2.imwrite("./result.jpg", gammaImg)